In [118]:
using InMemoryDatasets
import InMemoryDatasets as IMD
using BenchmarkTools
using TimerOutputs
using DLMReader

In [207]:
include("../src/main.jl")

cartesianjoin (generic function with 1 method)

# Data

In [121]:
@show dsl = Dataset(xid = [111,222,333,444,222], 
                    x1 = [1,2,1,4,3], 
                    x2 = [-1.2,-3,2.1,-3.5,2.8],
                    #x3 = [Date("2019-10-03"), Date("2019-09-30"), Date("2019-10-04"), Date("2019-10-03"), Date("2019-10-03")],
                    x4 = ["abcd","efgh","ijkl","mnop","qrst"]);

@show dsr = Dataset(yid = [111,111,222,444,333],
                    y1 = [3,3,3,3,3],
                    y2 = [0,0, -3,1,2],
                    #y3 = [Date("2019-10-01"),Date("2019-10-01"), Date("2019-09-30"), Date("2019-10-05"), Date("2019-10-05")],
                    y4 = ["abc","abcd","efg","mnop","qrst"]);

dsl_big = Dataset(rand(10^3, 5), :auto);
modify!(dsl_big,[:x1,:x2,:x3] .=> byrow(x -> (x*100)));
modify!(dsl_big,[:x1,:x2,:x3] .=> byrow(round), [:x1,:x2,:x3] .=> byrow(Int));

dsr_big = Dataset(rand(10^3, 5), :auto);
modify!(dsr_big,[:x1,:x2,:x3] .=> byrow(x -> (x*100)));
modify!(dsr_big,[:x1,:x2,:x3] .=> byrow(round), [:x1,:x2,:x3] .=> byrow(Int));
function x2y(str)
  replace(str,'x' => 'y')
end
rename!(x2y,dsr_big);

dsl_big2 = Dataset(rand(10^4, 5), :auto);
modify!(dsl_big2,[:x1,:x2,:x3] .=> byrow(x -> (x*100)));
modify!(dsl_big2,[:x1,:x2,:x3] .=> byrow(round), [:x1,:x2,:x3] .=> byrow(Int));

dsr_big2 = Dataset(rand(10^4, 5), :auto);
modify!(dsr_big2,[:x1,:x2,:x3] .=> byrow(x -> (x*100)));
modify!(dsr_big2,[:x1,:x2,:x3] .=> byrow(round), [:x1,:x2,:x3] .=> byrow(Int));
function x2y(str)
  replace(str,'x' => 'y')
end
rename!(x2y,dsr_big2);

dsl = Dataset(xid = [111, 222, 333, 444, 222], x1 = [1, 2, 1, 4, 3], x2 = [-1.2, -3, 2.1, -3.5, 2.8], x4 = ["abcd", "efgh", "ijkl", "mnop", "qrst"]) = 5×4 Dataset
 Row │ xid       x1        x2        x4
     │ identity  identity  identity  identity
     │ Int64?    Int64?    Float64?  String?
─────┼────────────────────────────────────────
   1 │      111         1      -1.2  abcd
   2 │      222         2      -3.0  efgh
   3 │      333         1       2.1  ijkl
   4 │      444         4      -3.5  mnop
   5 │      222         3       2.8  qrst
dsr = Dataset(yid = [111, 111, 222, 444, 333], y1 = [3, 3, 3, 3, 3], y2 = [0, 0, -3, 1, 2], y4 = ["abc", "abcd", "efg", "mnop", "qrst"]) = 5×4 Dataset
 Row │ yid       y1        y2        y4
     │ identity  identity  identity  identity
     │ Int64?    Int64?    Int64?    String?
─────┼────────────────────────────────────────
   1 │      111         3         0  abc
   2 │      111         3         0  abcd
   3 │      222         3        -3  

# Working

In [ ]:
innerjoin(dsl,dsr,on = 1)  

## on syntax

In [ ]:
on = [1 => 1, 3 => 3]
on1 = ["xid" => "yid","x1"=>"y1"]
on2 = [(:xid => :yid) => isequal, (:x1 => :y1) => isless]
on3  = [:xid => :yid, :x1 => :y1 => isless]
#on4  = [:xid => :yid, :x1 => :y1]

In [ ]:
# 两边都有同名列，on = :id 用的
(typeof(on1) <: AbstractVector{<:Union{AbstractString, Symbol}})

In [ ]:
(typeof(on2) <: AbstractVector{<:Pair{<:Pair{<:IMD.ColumnIndex,<:IMD.ColumnIndex}, <:Function}})

In [ ]:
typeof(on3)

In [ ]:
on3  = [:xid => :yid, :x1 => :y1 => isless]
if (typeof(on3) <: AbstractVector{<:Pair{<:IMD.ColumnIndex, <:Any}})

  dsr_cols = Symbol[]
  equalon_dsr_cols = Symbol[]
  conditions = Function[]

  for element in map(x -> x.second, on3)
    if typeof(element) <: Pair
      !(element.second isa Function) && throw(ArgumentError("Need Function"))
      push!(dsr_cols, element.first)
      push!(conditions, element.second)
    elseif typeof(element) <: IMD.ColumnIndex
      push!(dsr_cols, element)
      push!(equalon_dsr_cols, element)
      push!(conditions, isequal)
    else
      
      throw(ArgumentError("error `on`"))
    end
  end

  onleft = IMD.multiple_getindex(IMD.index(dsl), map(x -> x.first, on))
  onright = IMD.multiple_getindex(IMD.index(dsr), dsr_cols)
  onright_equal = IMD.multiple_getindex(IMD.index(dsr), equalon_dsr_cols)

end

# Test

In [117]:
function fun1(x,y) 
  x <= y
end

fun1 (generic function with 1 method)

In [ ]:
innerjoin(dsl,dsr,on = [:xid=>:yid],multiple_match=true,obs_id=[true,true])

In [211]:
newds = cartesianjoin(dsl,dsr,on = [:xid=>:yid, :x1=>:y1=>fun1],
            multiple_match=[true,true],obs_id=[false,false]);
newds

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:            886μs /  98.0%           45.2KiB /  95.1%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 generate new ds            1    670μs   77.2%   670μs   35.6KiB   82.8%  35.6KiB
   2-2 left row _res        1    434μs   50.0%   434μs   17.5KiB   40.8%  17.5KiB
     2.2 fill _res          4    314μs   36.2%  78.5μs   14.6KiB   34.0%  3.65KiB
     2.1 init _res          4    100μs   11.5%  25.0μs      624B    1.4%     156B
     2.3 push _res          4   2.77μs    0.3%   692ns     80.0B    0.2%    20.0B
   2-5 newds dsr            1    146μs   16.8%   146μs   9.94KiB   23.1%  9.94KiB
     5.1 init _

5×9 Dataset
 Row │ xid       x1        x2        x4        y1        y2        y4        m ⋯
     │ identity  identity  identity  identity  identity  identity  identity  i ⋯
     │ Int64?    Int64?    Float64?  String?   Int64?    Int64?    String?   B ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │      111         1      -1.2  abcd             3         0  abc         ⋯
   2 │      111         1      -1.2  abcd             3         0  abc
   3 │      222         2      -3.0  efgh             3         0  abc
   4 │      333         1       2.1  ijkl             3         0  abc
   5 │      222         3       2.8  qrst             3         0  abc         ⋯
                                                               2 columns omitted

In [218]:
@time nds = cartesianjoin(dsl_big2,dsr_big2,on = [:x1=>:y1, :x2=>:y2=>fun1, :x3=>:y3=>isless],
check=false);
nrow(nds)

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:            848ms / 100.0%           19.4MiB / 100.0%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 generate new ds            1    720ms   84.9%   720ms   19.4MiB   99.9%  19.4MiB
   2-5 newds dsr            1    635ms   74.9%   635ms   8.56MiB   44.1%  8.56MiB
     5.3 fill _res          4    371ms   43.8%  92.8ms     64.0B    0.0%    16.0B
     5.2 fill _res          4    263ms   31.0%  65.7ms     64.0B    0.0%    16.0B
     5.1 init _res          4    258μs    0.0%  64.5μs   8.55MiB   44.1%  2.14MiB
   2-1 left row count       1   83.1ms    9.8%  83.1ms    117KiB    0.6%   117KiB
   2-2 left row

249098

In [219]:
nds = cartesianjoin(dsl_big2,dsr_big2,on = [:x1=>:y1, :x2=>:y2, :x3=>:y3],check=false);
nrow(nds)

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:            521ms / 100.0%            171KiB /  98.7%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 generate new ds            1    388ms   74.4%   388ms    157KiB   93.2%   157KiB
   2-5 newds dsr            1    301ms   57.8%   301ms   9.08KiB    5.4%  9.08KiB
     5.3 fill _res          2    178ms   34.3%  89.2ms     32.0B    0.0%    16.0B
     5.2 fill _res          2    122ms   23.5%  61.1ms     32.0B    0.0%    16.0B
     5.1 init _res          2   47.8μs    0.0%  23.9μs   1.91KiB    1.1%     976B
   2-1 left row count       1   85.9ms   16.5%  85.9ms    117KiB   69.6%   117KiB
   2-2 left row

97

In [214]:
@time nds = innerjoin(dsl_big2,dsr_big2,on = [:x1=>:y1, :x2=>:y2, :x3=>:y3],check=false);
nrow(nds)

  0.005906 seconds (1.73 k allocations: 683.766 KiB)


97

## multiple_match

In [ ]:
newds, dsl_count, total_length = 
  cartesianjoin(dsl,dsr,on = [:xid=>:yid=>isless],multiple_match=true)

In [ ]:
newds[!,[1,2,3,5,6,7,9,10,11]]


In [ ]:
total_length

In [ ]:
_create_multiple_match_col_cartesian(dsl_count, nothing, total_length)

In [ ]:
a = [1,2,3,5,3]

In [ ]:
count(==(1),a) >1 ? 1 : 0

In [ ]:
map(x->count(==(x),a) > 1 ? 1 : 0 ,a)

## obsid

In [ ]:
if obs_id[1]
  obs_id_name1 = Symbol(obs_id_name, "_left")
  # 初始化total length的missing
  obs_id_left = allocatecol(nrow(dsl) < typemax(Int32) ? Int32 : Int64, total_length)
  ## 和填充dsl用的同一个函数，研究一下
  _fill_oncols_left_table_inner!(obs_id_left, 1:nrow(dsl), ranges, new_ends, total_length; inbits = inbits, en2 = revised_ends, threads = threads)
  insertcols!(newds, ncol(newds)+1, obs_id_name1 => obs_id_left, unsupported_copy_cols = false, makeunique = makeunique)
end

if obs_id[2]
  obs_id_name2 = Symbol(obs_id_name, "_right")
  obs_id_right = allocatecol(T, total_length)
  _fill_right_cols_table_inner!(obs_id_right, idx, ranges, new_ends, total_length; inbits = inbits, en2 = revised_ends, threads = threads)
  insertcols!(newds, ncol(newds)+1, obs_id_name2 => obs_id_right, unsupported_copy_cols = false, makeunique = makeunique)
end

In [ ]:
obs_id_left = IMD.allocatecol(nrow(dsl) < typemax(Int32) ? Int32 : Int64, 4)

In [ ]:
fill_left_res(obs_id_left,1:nrow(dsl),[2, 1, 0, 1, 0],[2, 3, 3, 4, 4],true)
insertcols!(newds, ncol(newds)+1, obs_id_name1 => obs_id_left, unsupported_copy_cols = false, makeunique = true)

In [ ]:
insertcols!(newds, ncol(newds) + 1, :test => [1,2,3,5], unsupported_copy_cols=false, makeunique=true)

In [ ]:
newds.test

## mapformats
获取格式然后做值对比的时候套用format

In [ ]:
if mapformats[1]
  _fl = _date_value∘getformat(dsl, onleft[1])
end
if mapformats[2]
  _fr = _date_value∘getformat(dsr, onright[1])
end

l_idx = oncols_left[j]
if lmf
  _fl = getformat(dsl, l_idx)
else
  _fl = identity
end
if rmf
  _fr = getformat(dsr, r_idx)
else
  _fr = identity
end

if _fr == identity
  var_r_cpy = var_r
else
  var_r_cpy = map(_fr, var_r)
end

unwrap(x) = x
_flx = _fl(DataAPI.unwrap(x[i]))::T1

In [ ]:
function format1(x)
  x = 1
end
setformat!(dsl,2 => format1)
getformat(dsl, 2)

In [ ]:
map(format1, IMD._columns(dsl)[2])

## findall optimize

In [ ]:
function findall(t::Union{AbstractString,AbstractPattern}, s::AbstractString; overlap::Bool=false)
  found = UnitRange{Int}[]
  i, e = firstindex(s), lastindex(s)
  while true
      r = findnext(t, s, i)
      isnothing(r) && break
      push!(found, r)
      j = overlap || isempty(r) ? first(r) : last(r)
      j > e && break
      @inbounds i = nextind(s, j)
  end
  return found
end

In [74]:
flag = zeros(Bool,1000 * 1000);
sizeof(flag);
bf = BitVector(flag);
sizeof(bf);
@time findall(flag);
@time findall(bf);

  0.001077 seconds (1 allocation: 64 bytes)
  0.000022 seconds (1 allocation: 64 bytes)


In [77]:
flag[3:10] .= 1
flag[20] = 1
flag[30:35] .= 1
flag[50:500] .= 1;

In [80]:
@time findall(flag);
@time i, e = firstindex(flag), lastindex(flag);

  0.001176 seconds (1 allocation: 3.812 KiB)
  0.000008 seconds (2 allocations: 48 bytes)


In [56]:
@btime r = findnext(flag,10) 
findnext(flag,r+1)
r

  17.813 ns (0 allocations: 0 bytes)


10

In [54]:
@btime r = 10 + 1

  0.047 ns (0 allocations: 0 bytes)


11

In [94]:
i = 1
i = findnext(flag,i+=1)

3

In [114]:

@btime begin
  i, e = 1, lastindex(flag);
  found = []
  while true
    res = findnext(flag,i)  #  找到i索引之后的下一个符合条件的索引，没填条件默认true
    isnothing(res) && break # 找不到则break
    # j = true || isempty(r) ? first(r) : last(r) 有ranges的才要判断
    #j = r
      # 超出索引，break
    #@inbounds i = nextind(flag, j)  ## 纯纯的找下一个索引的位置
    indx = res%5 == 0 ? 5 : res%5 # r_len
    push!(found, indx)  # 处理逻辑,填充对应r的值
    #println(i,r)

    i = res + 1
    i > e && break
  end
end

  589.588 μs (6 allocations: 7.55 KiB)


In [ ]:

@btime begin
  i, e = firstindex(flag), lastindex(flag);
  found = []
  while true
    r = findnext(flag,i)  # 找到i索引之后的下一个符合条件的索引，没填条件默认true
    isnothing(r) && break # 找不到则break
    # j = true || isempty(r) ? first(r) : last(r) 有ranges的才要判断
    j = r
      # 超出索引，break
    @inbounds i = nextind(flag, j)  ## 纯纯的找下一个索引的位置

    push!(found, r)  # 处理逻辑,填充对应r的值
    #println(i,r)

    #i = r+1
    i > e && break
  end
end
found;

@btime dsr_idx = findall(flag)